In [41]:
# library Book management system


import tkinter as tk
import tkinter.ttk as ttk
import sqlite3
from tkinter import messagebox

# Drop the existing books table if it exists and create a new one
def create_table():
    conn = sqlite3.connect('books.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS books
                      (book_id INTEGER PRIMARY KEY,
                       title TEXT NOT NULL,
                       author TEXT NOT NULL,
                       rack_number TEXT NOT NULL,
                       book_count INTEGER NOT NULL)''')
    conn.commit()
    conn.close()

# Clear all input fields
def clear_entries():
    book_id_entry.delete(0, tk.END)
    title_entry.delete(0, tk.END)
    author_entry.delete(0, tk.END)
    rack_entry.delete(0, tk.END)

# Display all books in the Treeview
def display_books():
    tree.delete(*tree.get_children())
    conn = sqlite3.connect('books.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM books")
    rows = cursor.fetchall()
    conn.close()

    for row in rows:
        tree.insert("", tk.END, values=row)

# Insert a new book into the database
def insert_book():
    try:
        book_id = int(book_id_entry.get())
    except ValueError:
        messagebox.showerror("Error", "Book ID must be a number.")
        return

    title = title_entry.get()
    author = author_entry.get()
    rack_number = rack_entry.get()

    if title and author :
        conn = sqlite3.connect('books.db')
        cursor = conn.cursor()

        # Check if the same book already exists
        cursor.execute("SELECT * FROM books WHERE title = ? AND author = ? ", (title, author))
        existing_books = cursor.fetchall()

        if existing_books:
            # Update the book count if the book already exists in the same rack
            book_id = existing_books[0][0]
            book_count = existing_books[0][4] + 1
            cursor.execute("UPDATE books SET book_count = ? WHERE book_id = ?", (book_count, book_id))
        else:
            # Insert a new book record if it doesn't exist
            cursor.execute("INSERT INTO books (book_id, title, author, rack_number, book_count) VALUES (?, ?, ?, ?, ?)", (book_id, title, author, rack_number, 1))
        
        conn.commit()
        conn.close()
        clear_entries()
        display_books()
    else:
        messagebox.showerror("Error", "Please fill in all required fields.")

# Delete the selected book
def delete_book():
    selected_item = tree.selection()
    if selected_item:
        book_id = tree.item(selected_item[0], "values")[0]
        conn = sqlite3.connect('books.db')
        cursor = conn.cursor()
        cursor.execute("DELETE FROM books WHERE book_id=?", (book_id,))
        conn.commit()
        conn.close()
        display_books()
    else:
        messagebox.showerror("Error", "Please select a book to delete.")

# Search for a book by ID
def search_book():
    search_id = search_entry.get()
    try:
        book_id = int(search_id)
    except ValueError:
        messagebox.showerror("Error", "Please enter a valid book ID.")
        return
    
    conn = sqlite3.connect('books.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM books WHERE book_id = ?", (book_id,))
    rows = cursor.fetchall()
    conn.close()

    tree.delete(*tree.get_children())

    if rows:
        for row in rows:
            tree.insert("", tk.END, values=row)
    else:
        messagebox.showinfo("No Results", "No book found with the specified ID.")
 
# to exit from the window
def close_window():
    window.destroy()

# Create the database table if it doesn't exist
create_table()

# Set up the main Tkinter window
window = tk.Tk()
window.title("Library Management System")

def close_window():
    window.destroy()
    
# Title Frame
title_frame = tk.Frame(window)
title_frame.pack(side=tk.TOP, padx=10, pady=10)
txt_title = tk.Label(title_frame, text='LIBRARY MANAGEMENT SYSTEM', fg='blue', font=('Arial', 16, 'bold'))
txt_title.pack(side=tk.TOP)

# Left Frame for inputs and buttons
left_frame = tk.Frame(window)
left_frame.pack(side=tk.LEFT, padx=10, pady=10)

# Labels and entry fields
tk.Label(left_frame, text="Book ID:").grid(row=0, column=0, padx=5, pady=5, sticky="e")
book_id_entry = tk.Entry(left_frame)
book_id_entry.grid(row=0, column=1, padx=5, pady=5)

tk.Label(left_frame, text="Book Title:").grid(row=1, column=0, padx=5, pady=5, sticky="e")
title_entry = tk.Entry(left_frame)
title_entry.grid(row=1, column=1, padx=5, pady=5)

tk.Label(left_frame, text="Author:").grid(row=2, column=0, padx=5, pady=5, sticky="e")
author_entry = tk.Entry(left_frame)
author_entry.grid(row=2, column=1, padx=5, pady=5)

tk.Label(left_frame, text="Rack Number:").grid(row=3, column=0, padx=5, pady=5, sticky="e")
rack_entry = tk.Entry(left_frame)
rack_entry.grid(row=3, column=1, padx=5, pady=5)

# Buttons
tk.Button(left_frame, text="Display All", width=15, command=display_books).grid(row=4, column=0, padx=5, pady=5)
tk.Button(left_frame, text="Add Book", width=15, command=insert_book).grid(row=4, column=1, padx=5, pady=5)
tk.Button(left_frame, text="Delete Book", width=15, command=delete_book).grid(row=5, column=0, padx=5, pady=5)
tk.Button(left_frame, text="Exit", width=15, command=close_window).grid(row=5, column=1, padx=5, pady=5)

# Right Frame for table and search
right_frame = tk.Frame(window)
right_frame.pack(side=tk.RIGHT, padx=10, pady=10)

tk.Label(right_frame, text="Search by Book ID:").grid(row=0, column=0, padx=5, pady=5)
search_entry = tk.Entry(right_frame)
search_entry.grid(row=0, column=1, padx=5, pady=5)
tk.Button(right_frame, text="Search", width=10, command=search_book).grid(row=0, column=2, padx=5, pady=5)

# Tree view for displaying books 
tree = ttk.Treeview(right_frame, columns=("Book ID", "Title", "Author", "Rack Number", "Book Count"), show="headings")
tree.heading("Book ID", text="Book ID")
tree.heading("Title", text="Title")
tree.heading("Author", text="Author")
tree.heading("Rack Number", text="Rack Number")
tree.heading("Book Count", text="Book Count")
tree.column("Book ID", width=70, anchor=tk.CENTER)
tree.column("Title", width=150, anchor=tk.CENTER)
tree.column("Author", width=100, anchor=tk.CENTER)
tree.column("Rack Number", width=100, anchor=tk.CENTER)
tree.column("Book Count", width=100, anchor=tk.CENTER)
tree.grid(row=1, column=0, columnspan=3, padx=10, pady=10)

# Run the Tkinter event loop
window.mainloop()

In [43]:
from sqlite3 import *

def view():
    con = connect('books.db')
    cur = con.cursor()
    cur.execute("SELECT * FROM books")
    rows = cur.fetchall()
    con.close()
    
    return rows


In [45]:
view()

[(1, 'War and Peace', 'Leo Tolstoy', '001', 1),
 (2, 'My Story', 'Kamala Das', '001', 1),
 (3, 'The White Tiger', 'Aravind Adiga', '002', 1),
 (4, 'Q & A', 'Vikas Swarup', '002', 1),
 (5, 'Devdas', 'Sharath Chandra Chattopadhyay', '003', 1),
 (6, 'Godaan', 'Premchand', '002', 1),
 (7, 'The House of Spirits', 'Isabel Allende', '003', 1),
 (8, 'The Jungle Book', 'Rudyard Kipling', '001', 2),
 (9, 'Great Expectations', 'Charles Dickens', '003', 1),
 (10, 'Things Fall Apart', 'Chinua Achebe', '003', 1)]